**Run the following two cells before you begin.**

In [2]:
%autosave 10

Autosaving every 10 seconds


In [3]:
import pandas as pd
import numpy as np
import os 
import math
os.chdir("C:\\Users\\tirth\\Technocolabs\\project 2\\Data set")

______________________________________________________________________
**First, import your data set and define the sigmoid function.**
<details>
    <summary>Hint:</summary>
    The definition of the sigmoid is $f(x) = \frac{1}{1 + e^{-X}}$.
</details>

In [4]:
# Import the data set
df = pd.read_csv("cleaned_data.csv")

In [21]:
# Define the sigmoid function
def sigmoid(X):
    Y = 1 / (1 + np.exp(-X))
    return Y

**Now, create a train/test split (80/20) with `PAY_1` and `LIMIT_BAL` as features and `default payment next month` as values. Use a random state of 24.**

In [22]:
# Create a train/test split
x = df[['PAY_1','LIMIT_BAL']]
y = df['default payment next month']

from sklearn.model_selection import train_test_split as tts 
train_x, test_x, train_y, test_y = tts(x,y,test_size=0.2,random_state =96,stratify=y)

______________________________________________________________________
**Next, import LogisticRegression, with the default options, but set the solver to `'liblinear'`.**

In [23]:
from sklearn.linear_model import LogisticRegression as lr 
model = lr(random_state=96, solver="liblinear")

______________________________________________________________________
**Now, train on the training data and obtain predicted classes, as well as class probabilities, using the testing data.**

In [24]:
# Fit the logistic regression model on training data
model.fit(train_x,train_y)

LogisticRegression(random_state=96, solver='liblinear')

In [25]:
# Make predictions using `.predict()`
pred_y = model.predict(test_x)

In [26]:
train_p = model.predict_proba(train_x)
train_p

array([[0.7081656 , 0.2918344 ],
       [0.93029843, 0.06970157],
       [0.69387531, 0.30612469],
       ...,
       [0.6641663 , 0.3358337 ],
       [0.88551742, 0.11448258],
       [0.92091899, 0.07908101]])

In [38]:
# Find class probabilities using `.predict_proba()`
pred_prob = model.predict_proba(test_x)
pred_prob

array([[0.58442327, 0.41557673],
       [0.53404307, 0.46595693],
       [0.67920226, 0.32079774],
       ...,
       [0.60088402, 0.39911598],
       [0.58442327, 0.41557673],
       [0.79638195, 0.20361805]])

______________________________________________________________________
**Then, pull out the coefficients and intercept from the trained model and manually calculate predicted probabilities. You'll need to add a column of 1s to your features, to multiply by the intercept.**

In [28]:
# Add column of 1s to features


In [29]:
# Get coefficients and intercepts from trained model
coef = model.coef_
intercept = model.intercept_

In [31]:
manual_y_proba = sigmoid((coef * test_x.values) + intercept)
manual_y_proba

array([[0.5       , 0.41557673],
       [0.5       , 0.46595693],
       [0.5       , 0.32079774],
       ...,
       [0.5       , 0.39911598],
       [0.5       , 0.41557673],
       [0.5       , 0.20361805]])

______________________________________________________________________
**Next, using a threshold of `0.5`, manually calculate predicted classes. Compare this to the class predictions output by scikit-learn.**

In [32]:
# Manually calculate predicted classes
threshold = 0.5
manual_y_pred = []

for i in manual_y_proba[:, 1]:
    if i >= threshold:
        manual_y_pred.append(1)
    else:
        manual_y_pred.append(0)

manual_y_pred = np.asarray(manual_y_pred)        

manual_y_pred

array([0, 0, 0, ..., 0, 0, 0])

In [34]:
# Compare to scikit-learn's predicted classes
pos = pred_y == 1
neg = pred_y == 0
manual_pos = manual_y_pred == 1
manual_neg = manual_y_pred == 0
print(f"Positives = {sum(pos)} and Negatives = {sum(neg)} are predicted classes.")
print(f"Positives = {sum(manual_pos)} and Negatives = {sum(manual_neg)} are maually predicted classes.")

Positives = 0 and Negatives = 5333 are predicted classes.
Positives = 0 and Negatives = 5333 are maually predicted classes.


______________________________________________________________________
**Finally, calculate ROC AUC using both scikit-learn's predicted probabilities, and your manually predicted probabilities, and compare.**

In [39]:
# Use scikit-learn's predicted probabilities to calculate ROC AUC
from sklearn import metrics
print(f"predicted probabilities: {metrics.roc_auc_score(test_y, pred_prob[:, 1])}")

ROC AUC Score for predicted probabilities: 0.633534306286855


In [40]:
# Use manually calculated predicted probabilities to calculate ROC AUC
print(f"manually predicted probabilities: {metrics.roc_auc_score(test_y, manual_y_proba[:, 1])}")

manually predicted probabilities: 0.6258746906476287
